In [1]:
import spacy
import en_ner_bc5cdr_md as ner_model
from spacy import displacy
import pandas as pd

In [2]:
import os
print(os.getcwd())

mtsample_df = pd.read_csv('mtsamples.csv')

text_df = mtsample_df['transcription']
text_df = text_df.dropna()

text=""

for transcription in text_df:
    text = text + transcription

text = text[0:1000000]

c:\Users\cihat\Downloads\Extracting Medical Information From Clinical Text With NLP


In [3]:
nlp = ner_model.load()
doc = nlp(text)
image = displacy.render(doc, jupyter=True, style='ent')

c:\Users\cihat\Downloads\Extracting Medical Information From Clinical Text With NLP\venv\Lib\site-packages\spacy\language.py:2141: FutureWarning: Possible set union at position 6328
  deserializers["tokenizer"] = lambda p: self.tokenizer.from_disk(  # type: ignore[union-attr]


In [4]:
print("TEXT", "START", "END", "ENTITY TYPE")
for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

TEXT START END ENTITY TYPE
complaint of allergies 58 80 DISEASE
allergies 100 109 DISEASE
Claritin 200 208 CHEMICAL
Zyrtec 214 220 CHEMICAL
Tri-Cyclen 666 676 CHEMICAL
Zyrtec 1070 1076 CHEMICAL
loratadine 1134 1144 CHEMICAL
Nasonex 1237 1244 CHEMICAL
pains 1666 1671 DISEASE
knee pain 1682 1691 DISEASE
pain 1698 1702 DISEASE
pain 1719 1723 DISEASE
reflux disease.,PAST 1764 1784 DISEASE
Heart disease 2068 2081 DISEASE
stroke 2121 2127 DISEASE
diabetes 2152 2160 DISEASE
obesity 2170 2177 DISEASE
hypertension 2182 2194 DISEASE
allergic 2267 2275 DISEASE
chest pain 3004 3014 DISEASE
coronary artery disease 3030 3053 DISEASE
congestive heart failure 3055 3079 DISEASE
arrhythmia 3081 3091 DISEASE
atrial fibrillation 3093 3112 DISEASE
cholesterol 3130 3141 CHEMICAL
pulmonary embolism 3143 3161 DISEASE
CVA 3184 3187 CHEMICAL
venous insufficiency 3189 3209 DISEASE
thrombophlebitis 3211 3227 DISEASE
shortness of breath 3237 3256 DISEASE
COPD 3258 3262 DISEASE
emphysema 3264 3273 DISEASE
sleep apn

In [5]:
mtsample_df.dropna(subset=['transcription'], inplace=True)
mtsample_df_subset = mtsample_df.sample(n=100, replace=False, random_state=42)
mtsample_df_subset.info()
mtsample_df_subset.head()

<class 'pandas.core.frame.DataFrame'>
Index: 100 entries, 3162 to 3581
Data columns (total 6 columns):
 #   Column             Non-Null Count  Dtype 
---  ------             --------------  ----- 
 0   Unnamed: 0         100 non-null    int64 
 1   description        100 non-null    object
 2   medical_specialty  100 non-null    object
 3   sample_name        100 non-null    object
 4   transcription      100 non-null    object
 5   keywords           78 non-null     object
dtypes: int64(1), object(5)
memory usage: 5.5+ KB


,Unnamed: 0,description,medical_specialty,sample_name,transcription,keywords
3162,3162,Markedly elevated PT INR despite stopping Cou...,Hematology - Oncology,Hematology Consult - 1,"HISTORY OF PRESENT ILLNESS:, The patient is w...",NaN
1981,1981,Intercostal block from fourth to tenth interc...,Pain Management,Intercostal block - 1,"PREPROCEDURE DIAGNOSIS:, Chest pain secondary...","pain management, xylocaine, marcaine, intercos..."
1361,1361,The patient is a 65-year-old female who under...,SOAP / Chart / Progress Notes,Lobectomy - Followup,"HISTORY OF PRESENT ILLNESS: , The patient is a...","soap / chart / progress notes, non-small cell ..."
3008,3008,Construction of right upper arm hemodialysis ...,Nephrology,Hemodialysis Fistula Construction,"PREOPERATIVE DIAGNOSIS: , End-stage renal dise...","nephrology, end-stage renal disease, av dialys..."
4943,4943,Bronchoscopy with brush biopsies. Persistent...,Cardiovascular / Pulmonary,Bronchoscopy - 8,"PREOPERATIVE DIAGNOSIS: , Persistent pneumonia...","cardiovascular / pulmonary, persistent pneumon..."


In [6]:
from spacy.matcher import Matcher
pattern = [{'ENT_TYPE':'CHEMICAL'}, {'LIKE_NUM': True}, {'IS_ASCII': True}]
matcher = Matcher(nlp.vocab)
matcher.add("DRUG_DOSE", [pattern])
for transcription in mtsample_df_subset['transcription']:
    doc = nlp(transcription)
    matches = matcher(doc)
    for match_id, start, end in matches:
        string_id = nlp.vocab.strings[match_id]  # get string representation
        span = doc[start:end]  # the matched span adding drugs doses
        print(span.text, start, end, string_id,)
        #Add disease and drugs
        for ent in doc.ents:
            print(ent.text, ent.start_char, ent.end_char, ent.label_)

Xylocaine 20 mL 129 132 DRUG_DOSE
Chest pain 26 36 DISEASE
Chest pain 122 132 DISEASE
intercostal block 318 335 DISEASE
chest pain 388 398 DISEASE
Xylocaine 730 739 CHEMICAL
Marcaine 750 758 CHEMICAL
contusion 987 996 DISEASE
pain 1150 1154 DISEASE
Marcaine 0.25% 133 136 DRUG_DOSE
Chest pain 26 36 DISEASE
Chest pain 122 132 DISEASE
intercostal block 318 335 DISEASE
chest pain 388 398 DISEASE
Xylocaine 730 739 CHEMICAL
Marcaine 750 758 CHEMICAL
contusion 987 996 DISEASE
pain 1150 1154 DISEASE
Aspirin 81 mg 204 207 DRUG_DOSE
non-small cell lung cancer 114 140 DISEASE
barium 322 328 CHEMICAL
hernia 380 386 DISEASE
odynophagia 647 658 DISEASE
thoracic dysphagia 667 685 DISEASE
tenderness 829 839 DISEASE
DVT 918 921 DISEASE
weight loss 952 963 DISEASE
anorexia 965 973 DISEASE
fevers 975 981 DISEASE
chills 983 989 DISEASE
headaches 991 1000 DISEASE
aches 1006 1011 DISEASE
pains 1015 1020 DISEASE
cough 1022 1027 DISEASE
hemoptysis 1029 1039 DISEASE
shortness of breath 1041 1060 DISEASE
dyspne